In [1]:
import requests
import json

API_KEY= 'Your own Yelp API key'

url = "https://api.yelp.com/v3/businesses/search"

headers = {'Authorization': 'Bearer %s' % API_KEY}

terms = ['chinese food', 'japanese food', 'korean food', 'italian food', 'mexican food','american food','indian food']
location = 'Manhattan, NY'
limit = 50
data = dict.fromkeys(terms)
ids = []

for term in terms:
    data[term]=[]
    
    for offset in range(0, 1000, limit):
        url_params = {
            'term': term.replace(' ', '+'),
            'location': location.replace(' ', '+'),
            'limit': limit,
            'offset': offset
        }
    
        response = requests.request("GET", url, headers=headers, params=url_params)

        if response.status_code == 200:
            querys = response.json()['businesses']
            for query in querys:
                if query not in ids:
                    ids.append(query)
                    data[term].append(query)
                    print('num_restaurants: {}'.format(len(ids)), end="\r")
        elif response.status_code == 400:
            continue
            
print('num_restaurants: {}',format(len(ids)))
for term in terms:
    print('{}: {}'.format(term,len(data[term])))

num_restaurants: {} 5186
chinese food: 999
japanese food: 671
korean food: 589
italian food: 889
mexican food: 837
american food: 509
indian food: 692


In [2]:
data['chinese food'][0]

{'id': 'ogCC-lJJYnwXDvKGmKZ6Sw',
 'alias': 'woorijip-new-york-2',
 'name': 'Woorijip',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/NZcGFutBjf1frDkslh6urw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/woorijip-new-york-2?adjust_creative=LZLTsquQ2fcAgttiVIjIAQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LZLTsquQ2fcAgttiVIjIAQ',
 'review_count': 2645,
 'categories': [{'alias': 'korean', 'title': 'Korean'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.74747, 'longitude': -73.98648},
 'transactions': ['delivery', 'pickup'],
 'price': '$',
 'location': {'address1': '12 W 32nd St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10001',
  'country': 'US',
  'state': 'NY',
  'display_address': ['12 W 32nd St', 'New York, NY 10001']},
 'phone': '+12122441115',
 'display_phone': '(212) 244-1115',
 'distance': 1578.239664148041}

In [3]:
import requests
import json
import boto3
from decimal import Decimal

In [7]:
modify_data = {}
for term in terms:
    modify_data[term.split(' ')[0]] = []
    for value in data[term]:
        tmp = {}
        tmp['id'] = value['id']
        tmp['name'] = value['name']
        tmp['rating'] = Decimal(str(value['rating']))
        tmp['latitude'] = Decimal(str(value['coordinates']['latitude']))
        tmp['longitude'] = Decimal(str(value['coordinates']['longitude']))
        tmp['zipcode'] = value['location']['zip_code']
        tmp['address'] = ""
        for ad in value['location']['display_address']:
            tmp['address'] += ad + " "
        tmp['reviews'] = Decimal(str(value['review_count']))
        tmp['phone'] = value['display_phone']
        modify_data[term.split(' ')[0]].append(tmp)

In [9]:
len(modify_data)

7

In [10]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('yelp-restaurants')
print("Table Status: ", table.table_status)
for key in modify_data.keys():
    for value in modify_data[key]:
        response = table.put_item(Item=value)

Table Status:  ACTIVE


In [11]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

host = 'Your own ElasticSearch host' # For example, my-test-domain.us-east-1.es.amazonaws.com
region = 'us-east-1' # e.g. us-west-1

service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

es = Elasticsearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [13]:
es_data = []
for key in modify_data.keys():
    for value in modify_data[key]:
        tmp = {}
        tmp['id'] = value['id']
        tmp['cuisine'] = key
        es_data.append(tmp)

In [14]:
es_data[0]

{'id': 'ogCC-lJJYnwXDvKGmKZ6Sw', 'cuisine': 'chinese'}

In [15]:
for document in es_data:
    es.index(index="restaurants", doc_type="Restaurant",body=document)